In [1]:
from surprise import SVD
from surprise import accuracy
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
import pandas as pd

ratings = pd.read_csv('ani_list.csv')
ratings.to_csv('ani_surprise.csv', index=False, header=False)
reader = Reader(rating_scale=(0, 10))

In [2]:
# load_from_df사용해서 데이터프레임을 데이터셋으로 로드
# 인자에 userid-itemid-ratings 변수들이 포함된 데이터프레임형태로 넣어주면 됨!
data = Dataset.load_from_df(ratings[['user_id','anime_id','rating']],
                           reader=reader)

train, test = train_test_split(data, test_size=0.25, random_state=42)

algo = SVD(n_factors=10, random_state=42)
algo.fit(train)
predictions = algo.test(test)
accuracy.rmse(predictions)

RMSE: 3.0558


3.055832815057323

In [29]:
#실제 평점과 예측된 평점의 차이를 RMSE로 측정한다. 
#RMSE는 오차(E)를 제곱(S)해서 평균(M)낸 값의 제곱근(R)이다. RMSE가 작을 수록 예측이 잘 된 것이다.
pred = algo.test(test)

In [4]:
from surprise.dataset import DatasetAutoFolds

In [25]:
reader = Reader(line_format='user item rating', sep=',',
               rating_scale=(0, 10))

# DatasetAutoFolds 클래스를 사용해서 개별적으로 생성
# index와 header가 없는 상태로 재생성했던 ratings_surprise.csv파일에 기반
data_folds = DatasetAutoFolds(ratings_file='ani_surprise.csv',
                             reader=reader)

# # 위에서 개별적으로 생성한 csv파일을 학습데이터로 생성
trainset = data_folds.build_full_trainset()
algo = SVD(random_state=42)
algo.fit(trainset)

# 영화에 대한 정보 데이터 로딩
anime = pd.read_excel('anime1.xlsx')
ratings = pd.read_csv('ani_list.csv',encoding='cp949')
anime.rename(columns = {'MAL_ID':'anime_id'},inplace=True)

# 데이터 확인용 줄
# 특정 사용자 9번의 movieId를 추출해서 특정 영화에 대한 평점 있는지 확인
aniIds = ratings[ratings['user_id'] == 9]['anime_id']
if aniIds[aniIds == 292].count() == 0:
    print('user id=9인 사람은 movie id=292 대한 평점이 없음')
    
# 영화에 대한 정보 데이터에서 movieId가 42인 영화가 무엇인지 출력
anime[anime['anime_id'] == 292]



user id=9인 사람은 movie id=292 대한 평점이 없음


,anime_id,Name,Score,Genres,Type,Ranked,Favorites
268,292,Dear Boys,6.85,"Drama, Shounen, Sports",TV,4240,48


In [26]:
sample=ratings.tail(10)
ex = ratings.tail(10).anime_id
ex_id = ex.values

In [27]:
pre = []
for i in ex_id:
    uid = str(3579)
    mid = str(i)
    pre.append(round(algo.predict(uid, mid).est,1))
pre

[5.9, 6.4, 5.1, 8.1, 7.4, 6.8, 7.1, 7.0, 6.6, 6.7]

In [28]:
sample['predict']=pre
sample.index=['p1','p2','p3','p4','p5','p6','p7','p8','p9','p10']
sample

C:\Users\user\AppData\Local\Temp/ipykernel_7324/3026407135.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['predict']=pre


,user_id,anime_id,rating,predict
p1,3579,14829,6,5.9
p2,3579,12565,7,6.4
p3,3579,356,5,5.1
p4,3579,25537,9,8.1
p5,3579,33049,7,7.4
p6,3579,22297,7,6.8
p7,3579,28701,7,7.1
p8,3579,27821,6,7.0
p9,3579,10087,6,6.6
p10,3579,11741,7,6.7


In [16]:
def get_unseen_surprise(ratings, anime, user_id):
    # 특정 유저가 본 ani id들을 리스트로 할당
    seen_anime = ratings[ratings['user_id']==user_id]['anime_id'].tolist()
    # 모든 만화들의 ani id들 리스트로 할당
    total_anime = anime['anime_id'].tolist()
    
    # 모든 영화들의 movie id들 중 특정 유저가 본 movie id를 제외한 나머지 추출
    unseen_anime = [ani for ani in total_anime if ani not in seen_anime]
    print(f'특정 {user_id}번 유저가 본 만화 수: {len(seen_anime)}\n아직 안본 만화 개수: {len(unseen_anime)}\n전체 만화수: {len(total_anime)}')
    
    return unseen_anime

In [17]:
def recomm_ani(algo, user_id, unseen_anime, top_n=10):
    # 알고리즘 객체의 predict()를 이용해 특정 userId의 평점이 없는 영화들에 대해 평점 예측
    predictions = [algo.predict(str(user_id), str(anime_id)) for anime_id in unseen_anime]
    
    # predictions는 Prediction()으로 하나의 객체로 되어있기 때문에 예측평점(est값)을 기준으로 정렬해야함
    # est값을 반환하는 함수부터 정의. 이것을 이용해 리스트를 정렬하는 sort()인자의 key값에 넣어주자!
    def sortkey_est(pred):
        return pred.est
    
    # sortkey_est함수로 리스트를 정렬하는 sort함수의 key인자에 넣어주자
    # 리스트 sort는 디폴트값이 inplace=True인 것처럼 정렬되어 나온다. reverse=True가 내림차순
    predictions.sort(key=sortkey_est, reverse=True)
    # 상위 n개의 예측값들만 할당
    top_predictions = predictions[:top_n]
    
    # top_predictions에서 movie id, rating, movie title 각 뽑아내기
    top_ani_ids = [int(pred.iid) for pred in top_predictions]
    top_ani_ratings = [pred.est for pred in top_predictions]
    top_ani_titles = anime[anime.anime_id.isin(top_ani_ids)]['Name']
    # 위 3가지를 튜플로 담기
    # zip함수를 사용해서 각 자료구조(여기선 리스트)의 똑같은 위치에있는 값들을 mapping
    # zip함수는 참고로 여러개의 문자열의 똑같은 위치들끼리 mapping도 가능!
    top_ani_preds = [(ids, rating, title) for ids, rating, title in zip(top_ani_ids, top_ani_ratings, top_ani_titles)]
    
    return top_ani_preds

In [18]:
# 위에서 정의한 함수를 사용해 특정 유저의 추천 영화들 출력해보기
unseen_lst = get_unseen_surprise(ratings, anime, 3579)
top_ani_preds = recomm_ani(algo, 3579, unseen_lst,top_n=10)
print()
print('-'*3,'Top-10 추천만화 리스트','-'*3)

# top_movies_preds가 여러가지의 튜플을 담고 있는 리스트이기 때문에 반복문 수행
for top_ani in top_ani_preds:
    print(top_ani[2], ":", round(top_ani[1],2))

특정 3579번 유저가 본 만화 수: 66
아직 안본 만화 개수: 17434
전체 만화수: 17500

--- Top-10 추천만화 리스트 ---
Aa! Megami-sama!: Sorezore no Tsubasa : 9.16
Hachimitsu to Clover II : 9.13
Furiko : 9.07
Koe no Katachi : 9.07
One Piece Film: Gold : 9.04
Fate/kaleid liner Prisma?낹llya 3rei!! : 8.96
Kimi no Na wa. : 8.93
Violet Evergarden : 8.92
Ballroom e Youkoso : 8.91
Ani ni Tsukeru Kusuri wa Nai! 2 : 8.85
